## Calculate features from ICs

- Edge fraction
- High frequency content
- ...
- (features from paper)

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nilearn.image import load_img, threshold_img, math_img, resample_to_img
from scipy.ndimage.morphology import binary_erosion
from scipy.signal import periodogram
from os.path import join, pardir
sys.path.append(pardir)
from bids import BIDSLayout
from tqdm import tqdm

/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.6 support is deprecated and will be removed in release 0.10 of Nilearn. Consider switching to Python 3.8 or 3.9.
  _python_deprecation_warnings()


In [2]:
# Helper function
def get_comps(metainfo_dict): # runinfo = bids layout?? think it's only needed to retrieve files
    """
    ...
    """
    # retrieve files
    print(">> get_compt")
    mixmat = np.loadtxt(join(metainfo_dict['fullpath'], 'melodic_mix'))
    ica_nii_f = join(metainfo_dict['fullpath'], 'melodic_IC.nii.gz')
    comps_arr = load_img(ica_nii_f).get_fdata()
    print("mixmat: ", join(metainfo_dict['fullpath'], 'melodic_mix'))
    print("ica_nii_f: ", ica_nii_f)
    print("comps_arr: ", comps_arr.shape)
    return mixmat, comps_arr

# Helper function
def get_edge_mask(metainfo_dict, ds_layout): # runinfo = bids layout?? think it's only needed to retrieve files
    """
    ...
    """
    # retrieve fmriprep/func brainmask file
    brainmask_f_temp = ds_layout.get(
        scope='derivatives',
        return_type='filename',
        subject=metainfo_dict['subject'],
        session=metainfo_dict['session'],
        run=metainfo_dict['run'],
        task=metainfo_dict['task'],
        space=metainfo_dict['space'],
        desc='brain',
        suffix='mask',
        extension='nii.gz'
    )
    brainmask_f = brainmask_f_temp[0]
    # Not sure if I have correct file here, file from Oli's script is:
    # 'fmriprep/../sub-XX_acq-prescannormalized_rec-pydeface_label-CSF_probseg.nii.gz'
    if metainfo_dict['space'] == 'T1w':
        csf_anat_f_temp = ds_layout.get(
                #scope='fmriprep', -> seems not to work, don't know why
                return_type='filename',
                subject=metainfo_dict['subject'],
                #space=metainfo_dict['space'],
                label='CSF',
                suffix='probseg',
                extension='nii.gz'
        )
        print('csf_anat_f_temp: ',csf_anat_f_temp)
        print('csf_anat_f_temp[0]: ',csf_anat_f_temp[0])

        csf_anat_f = csf_anat_f_temp[0]
    else:
        csf_anat_f_temp = ds_layout.get(
                #scope='fmriprep', -> seems not to work, don't know why
                return_type='filename',
                subject=metainfo_dict['subject'],
                space=metainfo_dict['space'], # T1w seems not defined --> maybe this is the problem here?
                label='CSF',
                suffix='probseg',
                extension='nii.gz'
        )
        csf_anat_f = csf_anat_f_temp[0]
    csf_func = threshold_img(
        resample_to_img(csf_anat_f, brainmask_f, interpolation='linear'),
        threshold=1.
    )
    brainmask = load_img(brainmask_f).get_fdata()
    mask_img = math_img('img1 - img2', img1=brainmask_f, img2=csf_func)
    mask_arr = mask_img.get_fdata()
    # worked okayish with erosion iterations=2
    # -> what is erosion?
    edgefrac_thickness = int(2)
    ero_mask = binary_erosion(mask_arr, iterations=edgefrac_thickness).astype(int)
    edgemask = mask_arr - ero_mask
    return edgemask.astype(bool), brainmask.astype(bool)

# Edge fraction
def calc_edgefrac(comp_arr, edgemask, brainmask):
    return np.absolute(comp_arr[edgemask]).sum() / np.absolute(comp_arr[brainmask]).sum()

# High frequency content
def calc_hfc(timeseries, tr=1.5):
    """Calculate high frequency content for time series data. Tr can generally mean sampling rate in seconds."""
    nf = (1. / tr) * .5  # nyquist
    freqs, power = periodogram(timeseries, fs=1. / tr)
    relcumsum = np.cumsum(power) / power.sum()
    freqind = np.argmin(np.absolute(relcumsum - .5))
    hfc = freqs[freqind] / nf
    return hfc

In [3]:
# Hardcoded paths 1/2
bidsdata_dir = '/LOCAL/jzerbe/emotion_category/ds003548'#'/LOCAL/jzerbe/faces_vs_houses/ds002938'
base_dir = '/LOCAL/jzerbe/emotion_category/' #'/LOCAL/jzerbe/temp_results' # not needed?
melodic_base_dir = join(bidsdata_dir, 'derivatives', 'melodic')
# Create layout
ds_layout = BIDSLayout(bidsdata_dir, derivatives=True)
melodic_entities = ds_layout.get(scope='melodic', return_type='filename', suffix='IC', extension='nii.gz')

In [4]:
melodic_entities

['/LOCAL/jzerbe/emotion_category/ds003548/derivatives/melodic/sub-01/sub-01_ses-None_task-emotionalfaces_run-1_space-MNI152NLin6Asym_melodic/melodic_IC.nii.gz',
 '/LOCAL/jzerbe/emotion_category/ds003548/derivatives/melodic/sub-01/sub-01_ses-None_task-emotionalfaces_run-1_space-MNI152NLin2009cAsym_melodic/melodic_IC.nii.gz',
 '/LOCAL/jzerbe/emotion_category/ds003548/derivatives/melodic/sub-01/sub-01_ses-None_task-emotionalfaces_run-1_space-T1w_melodic/melodic_IC.nii.gz',
 '/LOCAL/jzerbe/emotion_category/ds003548/derivatives/melodic/sub-01/sub-01_ses-None_task-emotionalfaces_run-2_space-MNI152NLin6Asym_melodic/melodic_IC.nii.gz',
 '/LOCAL/jzerbe/emotion_category/ds003548/derivatives/melodic/sub-01/sub-01_ses-None_task-emotionalfaces_run-2_space-MNI152NLin2009cAsym_melodic/melodic_IC.nii.gz',
 '/LOCAL/jzerbe/emotion_category/ds003548/derivatives/melodic/sub-01/sub-01_ses-None_task-emotionalfaces_run-2_space-T1w_melodic/melodic_IC.nii.gz',
 '/LOCAL/jzerbe/emotion_category/ds003548/derivati

In [17]:
# Main: Get dict with calculated features for each melodic run
results_dicts = []
i = 1
for entity in tqdm(melodic_entities, desc='iterating over runs'):
    # Cumbersome workaround to get correct filenames (TODO: better filenaming!)
    print("ENTITY", entity)
    print("**ITERATION** ", i)
    j = 1
    melodic_dir_split = entity.split('/')
    dir_name = melodic_dir_split[-2]
    metainfo_split = dir_name.split('_')
    metainfo_dict = {'subject':(metainfo_split[0])[4:], 'session':(metainfo_split[1])[4:],
                     'task':(metainfo_split[2])[5:], 'run':(metainfo_split[3])[4:],
                     'space':(metainfo_split[4])[6:], 'directory':melodic_dir_split[-2],
                     'fullpath':'/'.join(melodic_dir_split[:-1])}
    metainfo_dict['session'] = None if metainfo_dict['session'] == 'None' else metainfo_dict['session']
    metainfo_dict['run'] = None if metainfo_dict['run'] == 'None' else metainfo_dict['run']
    i += 1
    sys.exit()
    
    if metainfo_dict['subject'] in ['01', '02', '03', '04', '05', '06', '07', '08']:
        continue
        
    else:
    
        mixmat, comps_arr = get_comps(metainfo_dict)
        edgemask, brainmask = get_edge_mask(metainfo_dict, ds_layout)
        
        print("metainfo_dict['subject']", metainfo_dict['subject'])
        
    
        
        
        for comp_i in range(mixmat.shape[-1]):
            print("***INSIDE ITERATION*** ", j)
            results_dict = {'subject': metainfo_dict['subject'], 'session': metainfo_dict['session'],
                            'run': metainfo_dict['run'], 'task': metainfo_dict['task'],
                            'space': metainfo_dict['space']}
            print(results_dict)
            comp_arr = comps_arr[:, :, :, comp_i]
            comp_ts = mixmat[:, comp_i]
            print("comp_arr: ", comp_arr.shape)
            print("edgemask: ", edgemask.shape)
            print("brainmask: ", brainmask.shape)
            # Calculate edge fraction
            results_dict['edgefrac'] = calc_edgefrac(comp_arr, edgemask, brainmask)
            # Calculate high frequency content
            results_dict['hfc'] = calc_hfc(comp_ts)
            results_dicts.append(results_dict)
            j += 1
    

iterating over runs: 100%|██████████| 240/240 [00:00<00:00, 4332.57it/s]

ENTITY /LOCAL/jzerbe/emotion_category/ds003548/derivatives/melodic/sub-01/sub-01_ses-None_task-emotionalfaces_run-1_space-MNI152NLin6Asym_melodic/melodic_IC.nii.gz
**ITERATION**  1
ENTITY /LOCAL/jzerbe/emotion_category/ds003548/derivatives/melodic/sub-01/sub-01_ses-None_task-emotionalfaces_run-1_space-MNI152NLin2009cAsym_melodic/melodic_IC.nii.gz
**ITERATION**  2
ENTITY /LOCAL/jzerbe/emotion_category/ds003548/derivatives/melodic/sub-01/sub-01_ses-None_task-emotionalfaces_run-1_space-T1w_melodic/melodic_IC.nii.gz
**ITERATION**  3
ENTITY /LOCAL/jzerbe/emotion_category/ds003548/derivatives/melodic/sub-01/sub-01_ses-None_task-emotionalfaces_run-2_space-MNI152NLin6Asym_melodic/melodic_IC.nii.gz
**ITERATION**  4
ENTITY /LOCAL/jzerbe/emotion_category/ds003548/derivatives/melodic/sub-01/sub-01_ses-None_task-emotionalfaces_run-2_space-MNI152NLin2009cAsym_melodic/melodic_IC.nii.gz
**ITERATION**  5
ENTITY /LOCAL/jzerbe/emotion_category/ds003548/derivatives/melodic/sub-01/sub-01_ses-None_task-emot

In [ ]:
type(comps_arr)

In [ ]:
# Put results in dataframe
# sort after sapces (T1w, MNI)
results_df = pd.DataFrame(results_dicts)
results_df

In [ ]:
results_df['space'].unique()

In [ ]:
# Hardcoded paths 2/2
fig_path = '/LOCAL/jzerbe/code/ICA-fMRI/figures/'
fig_ds = 'emotion-category' # 'faces-vs-houses'
fig_space = 'MNI152NLin2009cAsym'
subplot_spaces = results_df[results_df['space'] == fig_space]

In [ ]:
# Visualize hfc
#hfc_plt = results_df['hfc'].hist(bins=100)
hfc_plt = plt.hist(subplot_spaces['hfc'], bins=100)
plt.title(f'HFC | {fig_ds} | {fig_space}')
plt.savefig(fig_path + f'{fig_ds}_hfc_{fig_space}' + '.jpg')

In [ ]:
# Visualize edge fraction
edgefrac_plt = plt.hist(subplot_spaces['edgefrac'], bins=100)
plt.title(f'Edge Fraction | {fig_ds} | {fig_space}')
plt.savefig(fig_path + f'{fig_ds}_edgefrac_{fig_space}' + '.jpg')

# Oli explores calculating ALT features

In [ ]:
import numpy as np
from nilearn.plotting import plot_img
from nilearn.image import math_img
from nilearn.masking import intersect_masks, apply_mask

In [ ]:
comp_f = "/LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/melodic/sub-03/sub-03_ses-None_task-effort_run-None_space-T1w-melodic/stats/thresh_zstat100.nii.gz"
aseg_f = '/LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-03/func/sub-03_task-effort_space-T1w_desc-aseg_dseg.nii.gz'
bmask_f = '/LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-03/func/sub-03_task-effort_space-T1w_desc-brain_mask.nii.gz'

In [ ]:
def aseg2gm(aseg_f):
    gm_left = math_img('img == 3', img=aseg_f)
    gm_right = math_img('img == 42', img=aseg_f)
    gm = intersect_masks([gm_left, gm_right], threshold=0, connected=False)
    return gm
gm = aseg2gm(aseg_f)

In [ ]:
def calc_gm_prop(comp_f, aseg_f, bmask_f):
    # get gm mask
    gm = aseg2gm(aseg_f)
    # count significant voxels in grey matter
    comp_gm = apply_mask(comp_f, gm)
    nsig_gm = np.sum(comp_gm > 0.)
    # counts ignificant voxels in whole brain
    comp_brain = apply_mask(comp_f, bmask_f)
    nsig_brain = np.sum(comp_brain > 0.)
    #return ratio
    gm_prop = nsig_gm / nsig_brain
    return gm_prop

In [ ]:
gm_prop

In [ ]:
# 